In [4]:
import numpy as np
import scipy.io as sio
import os
import copy

In [2]:
def count_leading_nans(arr):
    count = 0
    for val in arr:
        if np.isnan(val):
            count += 1
        else:
            break
    return count

def convert_sync(old_calib_path, new_calib_name):
    old_calib_data = sio.loadmat(old_calib_path)
    sync = old_calib_data['sync']
    new_sync = copy.deepcopy(sync)

    dff = 'data_frame'
    sid = 'data_sampleID'

    for camera in range(6):
        old_data_sampleID = sync[camera][0][sid][0][0][0]

        # # change all sampleID to actually one, note that this starts with 1, instead of 0. the old one started with 0
        lenn = len(old_data_sampleID)
        new_data_sampleID = list(range(1,lenn+1))
        new_sync[camera][0][sid][0][0][0] = new_data_sampleID

        # now, if sampleID starts with nan, then we need to adjust the new data_frame for it. if it is not nan, then it stays the same
        if old_data_sampleID[0]==0:
            # print(f'camera {camera+1} does not have nan, remains unchanged')
            continue
        else:
            nan_counts = count_leading_nans(old_data_sampleID)
            # print(f'camera {camera+1} has nan count of {nan_counts}, now starts with {nan_counts+1}')
            new_data_frame = list(range(nan_counts+1,lenn+1))
            new_sync[camera][0][dff][0][0] = np.array(new_data_frame)
    
    base_path = os.path.dirname(old_calib_path)
    save_path = os.path.join(base_path, new_calib_name)

    old_calib_data['sync'] = new_sync
    sio.savemat(save_path, old_calib_data)
    print('converted sync data saved to:', save_path)


In [5]:
old_calib_path = '/home/lq53/mir_data/sync_test/rawvid_mirstop_30s_light_3/timdataframe_sampleid_rescue/calib_convert_test/pos_synced_30s_label3d_dannce.mat'
new_calib_name = 'func_test_converted_label3d_dannce.mat'
convert_sync(old_calib_path, new_calib_name)

converted sync data saved to: /home/lq53/mir_data/sync_test/rawvid_mirstop_30s_light_3/timdataframe_sampleid_rescue/calib_convert_test/func_test_converted_label3d_dannce.mat


In [8]:
def replace_elements(A, B, C):
    #replace element of array in C, from A to B. but this method would not handle any nan values in the label....
    # Flatten the input array C to make indexing straightforward
    C_flat = C.flatten()
    
    # Create a dictionary to map values in A to their indices
    value_to_index = {value: idx for idx, value in enumerate(A)}
    
    # Iterate over the flattened array and replace values
    for i in range(len(C_flat)):
        if C_flat[i] in value_to_index:
            index_in_A = value_to_index[C_flat[i]]
            C_flat[i] = B[index_in_A]
    
    # Reshape the flattened array back to the original shape of C
    C = C_flat.reshape(C.shape)
    
    return C


def convert_label(old_calib_path, new_calib_path, label_path, new_label_name):
    old_calib_data = sio.loadmat(old_calib_path)
    new_calib_data = sio.loadmat(new_calib_path)
    label_data = sio.loadmat(label_path)

    dff = 'data_frame'
    sid = 'data_sampleID'

    labels = label_data['labelData']
    new_sync = new_calib_data['sync']
    old_sync = old_calib_data['sync']

    for camera_idx in range(6):
        old_sync_sid = old_sync[camera_idx][0][sid][0][0][0]

        if old_sync_sid[0]==0:
            good_label_sid = labels[camera_idx][0][sid][0][0]
            valid_label_sid = [x + 1 for x in good_label_sid]

    for camera_idx in range(6):
        old_sync_sid = old_sync[camera_idx][0][sid][0][0][0]
        new_label_sid = valid_label_sid
        
        if old_sync_sid[0]==0:
            labels[camera_idx][0][sid][0][0] = new_label_sid
            continue
        # then convert all incidences in the data 
        label_dff = labels[camera_idx][0][dff][0][0]
        # label_sid = labels[camera_idx][0][sid][0][0]

        old_sync_dff = old_sync[camera_idx][0][dff][0][0][0]
        

        new_sync_dff = new_sync[camera_idx][0][dff][0][0][0]
        # new_sync_sid = new_sync[camera_idx][0][sid][0][0][0]

        new_label_dff = replace_elements(old_sync_dff, new_sync_dff, label_dff)
        # replace_elements(old_sync_sid, new_sync_sid, label_sid)

        labels[camera_idx][0][dff][0][0] = new_label_dff
        labels[camera_idx][0][sid][0][0] = new_label_sid




    # step one: replace sync:
    label_data['sync'] = new_calib_data['sync']
    label_data['labelData'] = labels
 


    base_path = os.path.dirname(label_path)
    save_path = os.path.join(base_path, new_label_name)

    sio.savemat(save_path, label_data)
    print('converted label data saved to:', save_path) 

In [9]:
old_calib_path = '/home/lq53/mir_data/sync_test/rawvid_mirstop_30s_light_3/timdataframe_sampleid_rescue/calib_convert_test/pos_synced_30s_label3d_dannce.mat'
new_calib_path = '/home/lq53/mir_data/sync_test/rawvid_mirstop_30s_light_3/timdataframe_sampleid_rescue/calib_convert_test/converted_label3d_dannce.mat'
label_path = '/home/lq53/mir_data/sync_test/rawvid_mirstop_30s_light_3/timdataframe_sampleid_rescue/mir_sampleid_4framelabel_test_20240802_173848_Label3D_dannce.mat'
new_label_name = 'func_test_reverted_label_Label3D_dannce.mat'

convert_label(old_calib_path, new_calib_path, label_path, new_label_name)

converted label data saved to: /home/lq53/mir_data/sync_test/rawvid_mirstop_30s_light_3/timdataframe_sampleid_rescue/func_test_reverted_label_Label3D_dannce.mat
